In [1]:
# STANDARD
from pprint import pprint

# PyPI
from bs4 import BeautifulSoup as BS
import requests
from tqdm import tqdm

# LOCAL
from interface_db import db_interface_sqlite as data
from interface_website import website_tools as tools
from interface_website import website_patheos as patheos

In [2]:
with data.database() as db:
    db.create_tables()
    db.insert_update_site_pages(1625, 149)
    #result = db.execute(f"SELECT number FROM site_pages WHERE site_id = 47")
    #print(result[0][0])
    #for i in result:
    #    results = patheos.fetch_and_insert_blogs(i[0])
    #    print(f'Inserted: {results.inserted} | Not Inserted: {results.not_inserted} | Errors: {results.exceptions}')

In [3]:
patheos.insert_website('Patheos Blogs', 'https://patheos.com/blogs')
results = patheos.fetch_and_insert_categories('Patheos Blogs')
print(results)
with data.database() as db:
    result = db.execute("SELECT name FROM categories")
    for i in result:
        results = patheos.fetch_and_insert_blogs(i[0])
        print(f'Inserted: {results.inserted} | Not Inserted: {results.not_inserted} | Errors: {results.exceptions}')

Inserting Patheos Blogs - https://patheos.com/blogs
Insert successful. Name: Patheos Blogs, URL: https://patheos.com/blogs
Pulling categories for Patheos Blogs
https://www.patheos.com/buddhist-blogs
buddhist-blogs Inserted: 5 | Not Inserted: 0 | Errors: 0
https://www.patheos.com/catholic-blogs
catholic-blogs Inserted: 89 | Not Inserted: 0 | Errors: 0
https://www.patheos.com/contemplative-blogs
contemplative-blogs Inserted: 8 | Not Inserted: 0 | Errors: 0
https://www.patheos.com/evangelical-blogs
evangelical-blogs Inserted: 84 | Not Inserted: 0 | Errors: 0
https://www.patheos.com/general-christian-blogs
general-christian-blogs Inserted: 18 | Not Inserted: 0 | Errors: 0
https://www.patheos.com/hindu-blogs
hindu-blogs Inserted: 4 | Not Inserted: 0 | Errors: 0
https://www.patheos.com/jewish-blogs
jewish-blogs Inserted: 6 | Not Inserted: 0 | Errors: 0
https://www.patheos.com/latter-day-saint-blogs
latter-day-saint-blogs Inserted: 17 | Not Inserted: 0 | Errors: 0
https://www.patheos.com/musl

In [4]:
with data.database() as db:
    result = db.execute("SELECT * FROM blogs WHERE url = 'https://www.patheos.com/blogs/jesuscreed'")
    print(result)
    result = db.execute(f"SELECT number FROM site_pages WHERE site_id = 149")
    print(result)

[(149, 'by Scot McKnight', 'Jesus Creed', 'https://www.patheos.com/blogs/jesuscreed', 4, '2020-04-21', 'default', '2020-04-21', 'default')]
[(1625,)]


In [5]:
import sys

number_of_new_pages = patheos.number_of_blog_pages('Jesus Creed')
print(f'\nNew Pages: {number_of_new_pages}')

1640
New Pages: 15


In [7]:
results_l = patheos.scrape_posts_on_page('https://www.patheos.com/blogs/jesuscreed/page/3')
posts_dict = {}

results = tools.insert_results()

for i in results_l:
    print(i)
    post = data.post(i, 149)
    post = patheos.scrape_post(post, 'no')
    posts_dict[post.url] = post
    if tools.check_url_new('posts', post.url) == True:
        with data.database() as db:
            db.insert_post(post)
            results.inserted += 1
    else:
        # <placeholder for logging>
        results.not_inserted += 1
    
print(results.inserted, results.not_inserted)

https://www.patheos.com/blogs/jesuscreed/2020/01/01/prayer-for-new-years-day/
https://www.patheos.com/blogs/jesuscreed/2019/12/31/face-the-differences-rjs/
https://www.patheos.com/blogs/jesuscreed/2019/12/31/temple-then-temple-now/
https://www.patheos.com/blogs/jesuscreed/2019/12/31/humor-as-theology/
https://www.patheos.com/blogs/jesuscreed/2019/12/30/myth-singleness-is-a-special-calling/
https://www.patheos.com/blogs/jesuscreed/2019/12/30/a-b-simpson-and-me/
https://www.patheos.com/blogs/jesuscreed/2019/12/29/first-sunday-after-christmas-6/
https://www.patheos.com/blogs/jesuscreed/2019/12/27/a-single-man-on-gender-roles-and-the-church/
https://www.patheos.com/blogs/jesuscreed/2019/12/26/partly-sunny-partly-cloudy/
https://www.patheos.com/blogs/jesuscreed/2019/12/26/mary-was-brave-rjs/
0 10
